# 🔮 CryptOracle - Sentiment-Aware Crypto Forecasting

**BiLSTM + Attention Mechanism for Multi-Cryptocurrency Price Prediction**

---

This notebook will:
1. Clone the CryptOracle repository from GitHub
2. Install all dependencies (including TensorFlow)
3. Generate synthetic crypto data
4. Train the BiLSTM + Attention model
5. Evaluate performance and visualize results

**Estimated Runtime:** ~10 minutes with GPU (15-20 min without)

**💡 Tip:** Enable GPU for faster training!
- Go to: **Runtime → Change runtime type → Hardware accelerator → GPU**

## 📦 Step 1: Clone Repository & Setup

In [ ]:
# Clone the CryptOracle repository
!git clone https://github.com/singhshreya5477/CryptOracle.git
%cd CryptOracle

# List files to verify
!ls -la

## 🔧 Step 2: Install Dependencies

In [ ]:
# Install required packages
!pip install -q -r requirements.txt

# Verify TensorFlow installation
import tensorflow as tf
print(f"✓ TensorFlow version: {tf.__version__}")
print(f"✓ GPU Available: {tf.config.list_physical_devices('GPU')}")

# Check other key packages
import pandas as pd
import numpy as np
import sklearn
print(f"✓ Pandas: {pd.__version__}")
print(f"✓ NumPy: {np.__version__}")
print(f"✓ Scikit-learn: {sklearn.__version__}")

## 📊 Step 3: Generate Sample Data

We'll create realistic synthetic Bitcoin data with:
- OHLCV (Open, High, Low, Close, Volume)
- Fear & Greed Index
- Market Capitalization
- Realistic volatility using GARCH model

In [ ]:
# Generate synthetic cryptocurrency data
!python generate_sample_data.py

# Preview the generated data
import pandas as pd
df = pd.read_csv('data/raw/BTC_raw.csv')
print(f"\n✓ Generated {len(df)} days of BTC data")
print(f"✓ Date range: {df['Date'].min()} to {df['Date'].max()}")
print(f"\nFirst few rows:")
display(df.head())
print(f"\nLast few rows:")
display(df.tail())
print(f"\nData statistics:")
display(df.describe())

## 🚀 Step 4: Train the Model

This will:
- Feature engineering (RSI, MACD, Bollinger Bands, etc.)
- Create sequences for LSTM
- Build BiLSTM + Attention architecture
- Train for 100 epochs (with early stopping)
- Evaluate on test set

In [ ]:
# Run the complete training pipeline
!python run_pipeline.py

## 📈 Step 5: View Results

In [ ]:
# Load and display evaluation metrics
import json
import matplotlib.pyplot as plt
import numpy as np

# Read evaluation metrics
with open('models/saved_models/BTC_evaluation_metrics.json', 'r') as f:
    metrics = json.load(f)

print("=" * 60)
print("🎯 MODEL PERFORMANCE METRICS")
print("=" * 60)
print(f"✓ R² Score:              {metrics['r2_score']:.4f}")
print(f"✓ RMSE:                  ${metrics['rmse']:.2f}")
print(f"✓ MAE:                   ${metrics['mae']:.2f}")
print(f"✓ MAPE:                  {metrics['mape']:.2f}%")
print(f"✓ Directional Accuracy:  {metrics['directional_accuracy']:.2f}%")
print("=" * 60)

# Load predictions
predictions = np.load('models/saved_models/BTC_test_predictions.npy')
actuals = np.load('models/saved_models/BTC_test_actuals.npy')

# Plot predictions vs actuals
plt.figure(figsize=(15, 6))
plt.plot(actuals, label='Actual Price', linewidth=2, alpha=0.8)
plt.plot(predictions, label='Predicted Price', linewidth=2, alpha=0.8)
plt.title('CryptOracle: BTC Price Predictions vs Actuals', fontsize=14, fontweight='bold')
plt.xlabel('Time Steps (Days)', fontsize=12)
plt.ylabel('Price ($)', fontsize=12)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Plot residuals
residuals = actuals - predictions
plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
plt.scatter(predictions, residuals, alpha=0.5)
plt.axhline(y=0, color='r', linestyle='--', linewidth=2)
plt.title('Residual Plot', fontsize=12, fontweight='bold')
plt.xlabel('Predicted Price ($)', fontsize=10)
plt.ylabel('Residuals ($)', fontsize=10)
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.hist(residuals, bins=50, edgecolor='black', alpha=0.7)
plt.axvline(x=0, color='r', linestyle='--', linewidth=2)
plt.title('Residual Distribution', fontsize=12, fontweight='bold')
plt.xlabel('Residual ($)', fontsize=10)
plt.ylabel('Frequency', fontsize=10)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n✓ Mean Residual: ${np.mean(residuals):.2f}")
print(f"✓ Std Residual:  ${np.std(residuals):.2f}")

## 📊 Step 6: Training History

In [ ]:
# Load training history
with open('models/saved_models/BTC_training_history.json', 'r') as f:
    history = json.load(f)

# Plot training curves
plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
plt.plot(history['loss'], label='Training Loss', linewidth=2)
plt.plot(history['val_loss'], label='Validation Loss', linewidth=2)
plt.title('Model Loss', fontsize=12, fontweight='bold')
plt.xlabel('Epoch', fontsize=10)
plt.ylabel('Loss (MSE)', fontsize=10)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(history['mae'], label='Training MAE', linewidth=2)
plt.plot(history['val_mae'], label='Validation MAE', linewidth=2)
plt.title('Mean Absolute Error', fontsize=12, fontweight='bold')
plt.xlabel('Epoch', fontsize=10)
plt.ylabel('MAE', fontsize=10)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n✓ Total Epochs Trained: {len(history['loss'])}")
print(f"✓ Best Val Loss: {min(history['val_loss']):.6f}")
print(f"✓ Best Val MAE: {min(history['val_mae']):.6f}")

## 🏗️ Step 7: Model Architecture

In [ ]:
# Load and display model architecture
from tensorflow.keras.models import load_model
import sys
sys.path.append('/content/CryptOracle')
from model import AttentionLayer

model = load_model('models/saved_models/BTC_model.h5', 
                   custom_objects={'AttentionLayer': AttentionLayer})

print("=" * 60)
print("🏗️ CRYPTORACLE MODEL ARCHITECTURE")
print("=" * 60)
model.summary()

# Count parameters
trainable_params = sum([tf.size(w).numpy() for w in model.trainable_weights])
print(f"\n✓ Total Trainable Parameters: {trainable_params:,}")

## 💾 Step 8: Download Results

Download trained model and metrics to your local machine

In [ ]:
# Create a zip file with all results
!zip -r cryptoracle_results.zip models/saved_models/ logs/

# Download the results
from google.colab import files
files.download('cryptoracle_results.zip')

print("✓ Results packaged and downloading...")
print("\nContents:")
print("  • Trained model (BTC_model.h5)")
print("  • Evaluation metrics (JSON)")
print("  • Training history (JSON)")
print("  • Test predictions and actuals (NPY)")
print("  • Training logs")

## 🎨 Step 9: Run Streamlit Dashboard (Optional)

**Note:** Streamlit doesn't display directly in Colab, but you can:
1. Download the code to your local machine
2. Run: `streamlit run dashboard.py`
3. Or use this cell to expose via ngrok (requires ngrok authtoken)

In [ ]:
# Install pyngrok for tunneling
!pip install -q pyngrok

# Instructions for setting up ngrok
print("To run the Streamlit dashboard in Colab:")
print("1. Sign up for free ngrok account: https://dashboard.ngrok.com/signup")
print("2. Get your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken")
print("3. Run the following commands:")
print("")
print("from pyngrok import ngrok")
print('ngrok.set_auth_token("YOUR_NGROK_TOKEN")')
print("public_url = ngrok.connect(8501)")
print('print(f"Streamlit dashboard: {public_url}")')
print("!streamlit run dashboard.py &")
print("")
print("Alternatively, run the dashboard locally after downloading the code.")

## 🎯 Next Steps

### Experiment with the Model:
1. **Try different cryptocurrencies:**
   ```python
   !python run_pipeline.py --symbol ETH
   ```

2. **Adjust hyperparameters** in `config.py`:
   - LSTM units
   - Sequence length
   - Learning rate

3. **Ablation studies:**
   ```python
   !python run_pipeline.py --ablation no_attention
   !python run_pipeline.py --ablation simple_lstm
   ```

### Update Your README:
Replace the placeholder metrics in your README with the actual results from this training!

### Deploy:
- Push the trained model to your GitHub repo
- Deploy the Streamlit dashboard on Streamlit Cloud (free!)
- Share your project on LinkedIn/Twitter

---

**📧 Questions?** Open an issue on GitHub: https://github.com/singhshreya5477/CryptOracle

**⭐ Like this project?** Give it a star on GitHub!